In [ ]:
from gc import collect;
from warnings import filterwarnings;
filterwarnings('ignore');
from IPython.display import clear_output;

!pip install -q --upgrade scipy;
!pip install -q category_encoders;

clear_output();

In [ ]:
from copy import deepcopy;
import pandas as pd;
import numpy as np;
import re;
from scipy.stats import mode, kstest, normaltest, shapiro, anderson, jarque_bera;
from collections import Counter;
from itertools import product;
from colorama import Fore, Style, init;
from warnings import filterwarnings;
filterwarnings('ignore');

from tqdm.notebook import tqdm;
import seaborn as sns;
import matplotlib.pyplot as plt;
from matplotlib.colors import ListedColormap as LCM;
%matplotlib inline

from pprint import pprint;

In [ ]:
from category_encoders import OrdinalEncoder, OneHotEncoder;
from sklearn.preprocessing import (RobustScaler, 
                                   MinMaxScaler, 
                                   StandardScaler, 
                                   FunctionTransformer as FT
                                  );
from sklearn.impute import SimpleImputer as SI;
from sklearn.model_selection import (RepeatedStratifiedKFold as RSKF, 
                                     StratifiedKFold as SKF,
                                     KFold, 
                                     RepeatedKFold as RKF, 
                                     cross_val_score);
from sklearn.inspection import permutation_importance;
from sklearn.feature_selection import mutual_info_classif, RFE;
from sklearn.pipeline import Pipeline, make_pipeline;
from sklearn.base import BaseEstimator, TransformerMixin;
from sklearn.compose import ColumnTransformer;

from sklearn.metrics import roc_auc_score,  make_scorer;
from xgboost import DMatrix, XGBClassifier;
from lightgbm import LGBMClassifier, log_evaluation, early_stopping;
from catboost import CatBoostClassifier, Pool;
from sklearn.ensemble import (RandomForestClassifier as RFC, 
                              ExtraTreesClassifier as ETC,
                              AdaBoostClassifier as ABC,
                              BaggingClassifier as BC,
                              HistGradientBoostingClassifier as HGBC,
                              GradientBoostingClassifier as GBC,
                              BaggingClassifier as BC,
                             );
from sklearn.neighbors import KNeighborsClassifier as KNNC;
from sklearn.linear_model import LogisticRegression as LC;
from sklearn.calibration import CalibrationDisplay as Clb;

import optuna;
from optuna import Trial, trial, create_study;
from optuna.samplers import TPESampler, CmaEsSampler;
optuna.logging.set_verbosity = optuna.logging.ERROR;

In [ ]:
sns.set({"axes.facecolor"       : "#ffffff",
         "figure.facecolor"     : "#ffffff",
         "axes.edgecolor"       : "#000000",
         "grid.color"           : "#ffffff",
         "font.family"          : ['Cambria'],
         "axes.labelcolor"      : "#000000",
         "xtick.color"          : "#000000",
         "ytick.color"          : "#000000",
         "grid.linewidth"       : 0.75,  
         "grid.linestyle"       : "--",
         "axes.titlecolor"      : '#0099e6',
         'axes.titlesize'       : 8.5,
         'axes.labelweight'     : "bold",
         'legend.fontsize'      : 7.0,
         'legend.title_fontsize': 7.0,
         'font.size'            : 7.5,
         'xtick.labelsize'      : 7.5,
         'ytick.labelsize'      : 7.5,        
        });
 
def PrintColor(text:str, color = Fore.BLUE, style = Style.BRIGHT):
    "Prints color outputs using colorama using a text F-string";
    print(style + color + text + Style.RESET_ALL); 

from sklearn import set_config; 
set_config(transform_output = "pandas");
pd.set_option('display.max_columns', 50);
pd.set_option('display.max_rows', 50);


In [ ]:
# 参数配置类以及用于调整和训练的CV策略
class CFG:
    # 数据准备
    version_nb         = 3;
    test_req           = "N";
    # GPU型号
    gpu_switch         = "OFF"; 
    # 随机数种子
    state              = 42;
    # 目标列名
    target             = 'defects';
    # 系列序号
    episode            = 23;
    # 比赛数据文件夹路径
    path               = f"/kaggle/input/playground-series-s3e{episode}/";
    # 原始数据文件路径
    orig_path          = f"/kaggle/input/software-defect-prediction/jm1.csv";
    
    # 是否打印各列基本信息
    dtl_preproc_req    = "Y";
    adv_cv_req         = "N";
    # 是否绘制各列分布情况图
    ftre_plots_req     = 'Y';
    # 是否绘制各列相关性图
    ftre_imp_req       = "Y";
    
    # 数据转换和缩放   
    # 是否加入原始数据
    conjoin_orig_data  = "Y";
    # 是否删除空值
    drop_nulls         = "Y";
    sec_ftre_req       = "N";
    # 是否进行数据缩放
    scale_req          = "N";
    # 即使scale_req=N，也要在此处保留一个值，用于线性模型
    scl_method         = "Robust"; 
    enc_method         = 'Label';
    OH_cols            = [];
    
    # 模型训练
    baseline_req       = "N";
    pstprcs_oof        = "N";
    # 是否将训练集规整到0-1之间（回归）
    pstprcs_train      = "N";
    # 是否将测试集规整到0-1之间（回归）
    pstprcs_test       = "Y";
    # 是否训练模型
    ML                 = "Y";
    # 是否将原始数据加入到CV环节
    use_orig_allfolds  = "N";
    # 交叉检验折数
    n_splits           = 10 ;
    # RSKF交叉检验时数据重复次数
    n_repeats          = 1 ;
    # 早停策略容忍步数
    nbrnd_erly_stp     = 100 ;
    # 交叉检验方法
    mdlcv_mthd         = 'RSKF';
    
    # 模型组合    
    # 是否使用模型组合
    ensemble_req       = "Y";
    # 模型组合交叉检验方法
    enscv_mthd         = "RSKF";
    # optuna搜索方向（最大化/最小化目标值）
    metric_obj         = 'maximize';
    # 搜索次数
    ntrials            = 10 if test_req == "Y" else 1000;
    
    # 打印规范
    # 表格打印规范
    grid_specs = {'visible': True, 'which': 'both', 'linestyle': '--', 
                           'color': 'lightgrey', 'linewidth': 0.75};
    # 标题打印规范
    title_specs = {'fontsize': 9, 'fontweight': 'bold', 'color': 'tab:blue'};

PrintColor(f"--> Configuration done!\n");

In [ ]:
# 定义要在整个代码中用于常见任务的函数

# 用于连续列的缩放器
all_scalers = {'Robust': RobustScaler(), 
               'Z': StandardScaler(), 
               'MinMax': MinMaxScaler()
              };
scaler      = all_scalers.get(CFG.scl_method);

# 定义常用CV策略:-
all_cv= {'KF'  : KFold(n_splits= CFG.n_splits, shuffle = True, random_state= CFG.state),
         'RKF' : RKF(n_splits= CFG.n_splits, n_repeats = CFG.n_repeats, random_state= CFG.state),
         'RSKF': RSKF(n_splits= CFG.n_splits, n_repeats = CFG.n_repeats, random_state= CFG.state),
         'SKF' : SKF(n_splits= CFG.n_splits, shuffle = True, random_state= CFG.state)
        };

# 定义对比指标
def ScoreMetric(ytrue, ypred)-> float:
    return roc_auc_score(ytrue, ypred);

# 将结果更正到0~1
def PostProcessPred(preds, post_process = "N"):
    if post_process == "Y":
        return np.clip(preds, a_min = 0.0, a_max= 1.0);
    else:
        return preds;

# 设计一个自定义记分器以用于cross_val_epredict和cross_val_score
myscorer = make_scorer(ScoreMetric, greater_is_better = True, needs_proba= True,);

In [ ]:
"""
读取数据集
处理原始数据
检查信息和说明
检查唯一值和null
初始特征
根据CV结果判定是否需要加入原始数据
"""
class Preprocessor():
    def __init__(self):
        self.train    = pd.read_csv(CFG.path + f"train.csv", index_col = 'id');
        self.test     = pd.read_csv(CFG.path + f"test.csv", index_col = 'id');
        self.target   = CFG.target ;
        self.original = pd.read_csv(CFG.orig_path);
        self.conjoin_orig_data = CFG.conjoin_orig_data;
        self.dtl_preproc_req   = CFG.dtl_preproc_req;
        
        self.sub_fl   = pd.read_csv(CFG.path + f"sample_submission.csv");
        
        PrintColor(f"Data shapes - train-test-original = {self.train.shape} {self.test.shape} {self.original.shape}");
        
        for tbl in [self.train, self.original, self.test]:
            tbl.columns = tbl.columns.str.replace(r"\(|\)","", regex = True);
        
        PrintColor(f"\nTrain set head", color = Fore.GREEN);
        display(self.train.head(5).style.format(precision = 3));
        PrintColor(f"\nTest set head", color = Fore.GREEN);
        display(self.test.head(5).style.format(precision = 3));
        PrintColor(f"\nOriginal set head", color = Fore.GREEN);
        display(self.original.head(5).style.format(precision = 3));
                 
        # 重置原始数据索引
        self.original.index = range(len(self.original));
        self.original.index+= max(self.test.index) + 1;
        self.original.index.name = 'id';
        
        # 更改原始数据列顺序以训练数据结构
        self.original = self.original.reindex(self.train.columns, axis=1);
    
    # 为train、test、original添加分数列
    def _AddSourceCol(self):
        self.train['Source'] = "Competition";
        self.test['Source']  = "Competition";
        self.original['Source'] = 'Original';
        
        self.strt_ftre = self.test.columns;
        return self;
    
    # 将数值列的字符串替换为空值
    def _CleanOrigNulls(self):
        for col in ['uniq_Op', 'uniq_Opnd', 'total_Op', 'total_Opnd', 'branchCount']:
            self.original.loc[self.original[col] == r"?",col] = np.NaN;
            self.original[col] = self.original[col].astype(np.float64);
        return self;
    
    # 数据基本信息展示
    def _CollateInfoDesc(self):
        if self.dtl_preproc_req == "Y":
            PrintColor(f"\n{'-'*20} Information and description {'-'*20}\n", color = Fore.MAGENTA);

            # Creating dataset information and description:
            for lbl, df in {'Train': self.train, 'Test': self.test, 'Original': self.original}.items():
                PrintColor(f"\n{lbl} description\n");
                display(df.describe(percentiles= [0.05, 0.25, 0.50, 0.75, 0.9, 0.95, 0.99]).\
                        transpose().\
                        drop(columns = ['count'], errors = 'ignore').\
                        drop([CFG.target], axis=0, errors = 'ignore').\
                        style.format(formatter = '{:,.2f}').\
                        background_gradient(cmap = 'Blues')
                       );

                PrintColor(f"\n{lbl} information\n");
                display(df.info());
                collect();
        return self;
    
    # 展示各列唯一值个数
    def _CollateUnqNull(self):
        if self.dtl_preproc_req == "Y":
            PrintColor(f"\nUnique and null values\n");
            _ = pd.concat([self.train[self.strt_ftre].nunique(), 
                           self.test[self.strt_ftre].nunique(), 
                           self.original[self.strt_ftre].nunique(),
                           self.train[self.strt_ftre].isna().sum(axis=0),
                           self.test[self.strt_ftre].isna().sum(axis=0),
                           self.original[self.strt_ftre].isna().sum(axis=0)
                          ], 
                          axis=1);
            _.columns = ['Train_Nunq', 'Test_Nunq', 'Original_Nunq', 
                         'Train_Nulls', 'Test_Nulls', 'Original_Nulls'
                        ];

            display(_.T.style.background_gradient(cmap = 'Blues', axis=1).\
                    format(formatter = '{:,.0f}')
                   );
            
        return self;
    
    # 开始对数据进行处理
    def DoPreprocessing(self):
        self._CleanOrigNulls();
        self._AddSourceCol();
        self._CollateInfoDesc();
        self._CollateUnqNull();
        
        return self; 
    
    # 加入原始数据
    def ConjoinTrainOrig(self):
        if self.conjoin_orig_data == "Y":
            PrintColor(f"Train shape before conjoining with original = {self.train.shape}");
            train = pd.concat([self.train, self.original], axis=0, ignore_index = True);
            PrintColor(f"Train shape after conjoining with original= {train.shape}");
            
            train = train.drop_duplicates();
            PrintColor(f"Train shape after de-duping = {train.shape}");
            
            train.index = range(len(train));
            train.index.name = 'id';
        
        else:
            PrintColor(f"We are using the competition training data only");
            train = self.train;
        return train;

In [ ]:
pp = Preprocessor();
pp.DoPreprocessing();

In [ ]:
# 在2个指定数据集之间执行对抗性CV
def Do_AdvCV(df1:pd.DataFrame, df2:pd.DataFrame, source1:str, source2:str):
    # 对每一列进行检验
    ftre = pp.test.select_dtypes(include = np.number).\
    drop(columns = ['id', "Source"], errors = 'ignore').columns;
    adv_cv = {};
    
    for col in ftre:
        # 随机数种子
        shuffle_state = np.random.randint(low = 10, high = 100, size= 1);

        full_df = \
        pd.concat([df1[[col]].assign(Source = source1), df2[[col]].assign(Source = source2)], 
                  axis=0, ignore_index = True).\
        sample(frac = 1.00, random_state = shuffle_state);

        full_df = full_df.assign(Source_Nb = full_df['Source'].eq(source2).astype(np.int8));

        # Checking for adversarial CV:-
        model = LGBMClassifier(random_state = CFG.state, max_depth = 6, learning_rate = 0.05);
        cv    = all_cv['SKF'];
        score = np.mean(cross_val_score(model, 
                                        full_df[[col]], 
                                        full_df.Source_Nb, 
                                        scoring= 'roc_auc', 
                                        cv     = cv)
                       );
        # 添加得分情况
        adv_cv.update({col: round(score, 4)});
        collect();
    
    del ftre;
    
    fig, ax = plt.subplots(1,1,figsize = (12, 5));
    pd.Series(adv_cv).plot.bar(color = 'tab:blue', ax = ax);
    ax.axhline(y = 0.60, color = 'red', linewidth = 2.75);
    ax.grid(**CFG.grid_specs); 
    plt.yticks(np.arange(0.0, 0.81, 0.05));
    plt.show();
    
# 进行对抗性CV
if CFG.adv_cv_req == "Y":
    PrintColor(f"\n---------- Adversarial CV - Train vs Original ----------\n", 
               color = Fore.MAGENTA);
    Do_AdvCV(df1 = pp.train, df2 = pp.original, source1 = 'Train', source2 = 'Original');
    
    PrintColor(f"\n---------- Adversarial CV - Train vs Test ----------\n", 
               color = Fore.MAGENTA);
    Do_AdvCV(df1 = pp.train, df2 = pp.test, source1 = 'Train', source2 = 'Test');
    
    PrintColor(f"\n---------- Adversarial CV - Original vs Test ----------\n", 
               color = Fore.MAGENTA);
    Do_AdvCV(df1 = pp.original, df2 = pp.test, source1 = 'Original', source2 = 'Test');   
    
if CFG.adv_cv_req == "N":
    PrintColor(f"\nAdversarial CV is not needed\n", color = Fore.RED);

In [ ]:
# （训练数据+原始数据）、测试集、不包含目标列的其他列名
train, test, strt_ftre = pp.ConjoinTrainOrig(), pp.test.copy(deep = True), deepcopy(pp.strt_ftre);
# 类别列
cat_cols  = test.select_dtypes(include = 'object').columns[:-1];
# 连续数值列
cont_cols = test.drop(columns = [], errors = 'ignore').select_dtypes(exclude = 'object').columns;

# 若删除训练集中的空值
if CFG.drop_nulls == "Y":
    train = train.dropna();
else: 
    pass;

PrintColor(f"\nCategory columns\n");
display(cat_cols);
PrintColor(f"\nContinuous columns\n");
display(np.array(cont_cols));
PrintColor(f"\nAll columns\n");
display(strt_ftre);

In [ ]:
# 绘制训练数据和原始数据的目标分布
if CFG.ftre_plots_req == "Y":
    
    fig, axes = plt.subplots(1,2, figsize = (12, 5), sharey = True, gridspec_kw = {'wspace': 0.35});

    for i, df in tqdm(enumerate([pp.train, pp.original]), "Target balance ---> "):
        ax= axes[i];
        a = df[CFG.target].value_counts(normalize = True);
        _ = ax.pie(x = a , labels = a.index.values, 
                   explode      = [0.0, 0.2], 
                   startangle   = 40, 
                   shadow       = True, 
                   colors       = ['#3377ff', '#66ffff'], 
                   textprops    = {'fontsize': 7, 'fontweight': 'bold', 'color': 'black'},
                   pctdistance  = 0.60, 
                   autopct = '%1.1f%%'
                  );
        df_name = 'Train' if i == 0 else "Original";
        _ = ax.set_title(f"\n{df_name} data\n", **CFG.title_specs);

    plt.tight_layout();
    plt.show();

In [ ]:
# 显示分布具体数值
if CFG.ftre_plots_req == "Y":
    fig, axes = plt.subplots(1,2, figsize = (12, 4), gridspec_kw = {'wspace': 0.2});
    
    for i, (lbl, df) in enumerate({"Train": pp.train, "Original": pp.original}.items()):
        ax = axes[i];
        c = ['#3377ff', '#6699cc'];
        df.groupby(CFG.target).size().plot.bar(ax = ax, color = c[i]);
        ax.set_title(f"Target interaction - {lbl} set", **CFG.title_specs);
        ax.set(xlabel = "");
        
    plt.tight_layout();
    plt.show()

In [ ]:
# 对比训练数据、测试数据、原始数据各特征分布差异
if CFG.ftre_plots_req == "Y":
    df = pd.concat([pp.train[cont_cols].assign(Source = 'Train'), 
                    pp.test[cont_cols].assign(Source = 'Test'),
                    pp.original[cont_cols].assign(Source = "Original")
                   ], 
                   axis=0, ignore_index = True
                  );
    
    fig, axes = plt.subplots(len(cont_cols), 4 ,figsize = (16, len(cont_cols) * 4.2), 
                             gridspec_kw = {'hspace': 0.35, 'wspace': 0.3, 'width_ratios': [0.80, 0.20, 0.20, 0.20]});
    
    for i,col in enumerate(cont_cols):
        ax = axes[i,0];
        sns.kdeplot(data = df[[col, 'Source']], x = col, hue = 'Source', 
                    palette = ['#0039e6', '#ff5500', '#00b300'], 
                    ax = ax, linewidth = 2.1
                   );
        ax.set_title(f"\n{col}", **CFG.title_specs);
        ax.grid(**CFG.grid_specs);
        ax.set(xlabel = '', ylabel = '');
        
        ax = axes[i,1];
        sns.boxplot(data = df.loc[df.Source == 'Train', [col]], y = col, width = 0.25,
                    color = '#33ccff', saturation = 0.90, linewidth = 0.90, 
                    fliersize= 2.25,
                    ax = ax);
        ax.set(xlabel = '', ylabel = '');
        ax.set_title(f"Train", **CFG.title_specs);
        
        ax = axes[i,2];
        sns.boxplot(data = df.loc[df.Source == 'Test', [col]], y = col, width = 0.25, fliersize= 2.25,
                    color = '#80ffff', saturation = 0.6, linewidth = 0.90, 
                    ax = ax); 
        ax.set(xlabel = '', ylabel = '');
        ax.set_title(f"Test", **CFG.title_specs);
        
        ax = axes[i,3];
        sns.boxplot(data = df.loc[df.Source == 'Original', [col]], y = col, width = 0.25, fliersize= 2.25,
                    color = '#99ddff', saturation = 0.6, linewidth = 0.90, 
                    ax = ax); 
        ax.set(xlabel = '', ylabel = '');
        ax.set_title(f"Original", **CFG.title_specs);
              
    plt.suptitle(f"\nDistribution analysis- continuous columns\n", **CFG.title_specs, 
                 y = 0.905, x = 0.50
                );
    plt.tight_layout();
    plt.show();

In [ ]:
# 特征间的偏度
skew_df = pd.DataFrame(index = strt_ftre[0:-1]);
for col, df in {"Train": pp.train, "Test": pp.test, "Original": pp.original}.items():
    skew_df = \
    pd.concat([skew_df, 
               df.drop(columns = [CFG.target, "Source", "id"], errors = "ignore").skew()],
              axis=1).rename({0: col}, axis=1);

PrintColor(f"\nSkewness across independent features\n");
display(skew_df.transpose().style.format(precision = 2).background_gradient("Pastel2"));

del skew_df;

In [ ]:
# 绘制训练数据、测试数据、原始数据，pearson和spearman相关系数热力图
def MakeCorrPlot(df: pd.DataFrame, data_label:str, figsize = (30, 9)):
    """
    This function develops the correlation plots for the given dataset
    """;
    
    fig, axes = plt.subplots(1,2, figsize = figsize, gridspec_kw = {'hspace': 0.2, 'wspace': 0.1},
                             sharey = True
                            );
    
    for i, method in enumerate(['pearson', 'spearman']):
        corr_ = df.drop(columns = ['id', 'Source'], errors = 'ignore').corr(method = method);
        ax = axes[i];
        sns.heatmap(data = corr_,  
                    annot= True,
                    fmt= '.2f', 
                    cmap = 'Blues',
                    annot_kws= {'fontweight': 'bold','fontsize': 6.75}, 
                    linewidths= 1.5, 
                    linecolor='white', 
                    cbar= False, 
                    mask= np.triu(np.ones_like(corr_)),
                    ax= ax
                   );
        ax.set_title(f"\n{method.capitalize()} correlation- {data_label}\n", **CFG.title_specs);
        
    collect();
    print();

# Implementing correlation analysis:-
if CFG.ftre_imp_req == "Y":
    for lbl, df in {"Train": pp.train, 
                    "Test": pp.test, 
                    "Original": pp.original
                   }.items():
        MakeCorrPlot(df = df, data_label = lbl, figsize = (22,6.5));

In [ ]:
# 此类使用简单交互为现有数据添加特征
class Xformer(TransformerMixin, BaseEstimator):
    def __init__(self): 
        pass
    
    def fit(self, X, y= None, **params):
        return self;
    
    # 将特征添加到现有数据中
    def transform(self, X, y= None, **params):
        df    = X.copy(); 
        
        df['mean_bnv']         = (df['n'] + df['v'] + df['b']) /3;
        df['mean_uniqOpOpend'] = (df['uniq_Op'] + df['uniq_Opnd']) /2;
        df['mean_totOpOpend']  = (df['total_Op'] + df['total_Opnd']) /2;
        df['mean_brcntvg']     = (df['branchCount'] + df['vg']) / 2;
        
        self.op_cols = df.columns;  
        return df;
    
    def get_feature_names_in(self, X, y=None, **params): 
        return self.ip_cols;    
    
    def get_feature_names_out(self, X, y=None, **params): 
        return self.op_cols;

In [ ]:
# 数据缩放类
class Scaler(TransformerMixin, BaseEstimator):
    def __init__(self, scl_method: str, scale_req: str, scl_cols):
        self.scl_method = scl_method;
        self.scale_req  = scale_req;
        self.scl_cols   = scl_cols;
    
    # 计算训练集缩放参数
    def fit(self,X, y=None, **params):        
        self.params          = X[self.scl_cols].describe(percentiles = [0.25, 0.50, 0.75]).drop(['count'], axis=0).T;
        self.params['iqr']   = self.params['75%'] - self.params['25%'];
        self.params['range'] = self.params['max'] - self.params['min'];
        
        return self;
    
    # 使用特定方法缩放列
    def transform(self,X, y=None, **params):  
        df = X.copy();
        if self.scale_req == "Y":
            if CFG.scl_method == "Z":
                df[self.scl_cols] = (df[self.scl_cols].values - self.params['mean'].values) / self.params['std'].values;
            elif CFG.scl_method == "Robust":
                df[self.scl_cols] = (df[self.scl_cols].values - self.params['50%'].values) / self.params['iqr'].values;
            elif CFG.scl_method == "MinMax":
                df[self.scl_cols] = (df[self.scl_cols].values - self.params['min'].values) / self.params['range'].values;
        else:
            PrintColor(f"Scaling is not needed", color = Fore.RED);
    
        return df;
    

In [ ]:
PrintColor(f"\n{'='* 20} Data transformation {'='* 20} \n");

ytrain = train[CFG.target].astype(np.int8);
Xtrain = train.drop(columns = [CFG.target]);
Xtest  = test.copy(deep = True);

PrintColor(f"\n---> Train data\n");
display(Xtrain.head(5).style.format(precision = 2));
PrintColor(f"\n---> Test data\n");
display(Xtest.head(5).style.format(precision = 2));

xform = Pipeline(steps = [("Xform", Xformer())]);
Xtrain = xform.fit_transform(Xtrain, ytrain);
Xtest  = xform.transform(Xtest);

PrintColor(f"\n---> Train data columns after data pipeline\n");
pprint(Xtrain.columns);

PrintColor(f"\n---> Test data columns after data pipeline\n");
pprint(Xtest.columns);

PrintColor(f"\n---> Train-test shape after pipeline = {Xtrain.shape} {Xtest.shape}");

In [ ]:
# 各模型参数
Mdl_Master = \
{'CBC': CatBoostClassifier(**{'task_type'           : "GPU" if CFG.gpu_switch == "ON" else "CPU",
                              'objective'           : 'Logloss',
                              'loss_function'       : 'Logloss',
                              'eval_metric'         : 'AUC',
                              'bagging_temperature' : 0.20,
                              'colsample_bylevel'   : 0.75,
                              'iterations'          : 1000,
                              'learning_rate'       : 0.08,
                              'od_wait'             : 14,
                              'max_depth'           : 6,
                              'l2_leaf_reg'         : 0.85,
                              'min_data_in_leaf'    : 15,
                              'random_strength'     : 0.70, 
                              'max_bin'             : 120,
                              'verbose'             : 0,
                              'use_best_model'      : True,
                           }
                         ), 

  'LGBM1C': LGBMClassifier(**{'device'           : "gpu" if CFG.gpu_switch == "ON" else "cpu",
                             'objective'         : 'binary',
                             'metric'            : 'auc',
                             'boosting_type'     : 'gbdt',
                             'random_state'      : CFG.state,
                             'colsample_bytree'  : 0.5,
                             'subsample'         : 0.65,
                             'learning_rate'     : 0.065,
                             'max_depth'         : 6,
                             'n_estimators'      : 1000,
                             'num_leaves'        : 110,                    
                             'reg_alpha'         : 0.01,
                             'reg_lambda'        : 1.75,
                             'verbose'           : -1,
                          }
                       ),
 
   'LGBM2C': LGBMClassifier(**{'device'           : "gpu" if CFG.gpu_switch == "ON" else "cpu",
                              'objective'         : 'binary',
                              'metric'            : 'auc',
                              'boosting_type'     : 'goss',
                              'random_state'      : CFG.state,
                              'colsample_bytree'  : 0.50,
                              'subsample'         : 0.70,
                              'learning_rate'     : 0.0625,
                              'max_depth'         : 6,
                              'n_estimators'      : 1000,
                              'num_leaves'        : 110,                    
                              'reg_alpha'         : 0.0001,
                              'reg_lambda'        : 2.0,
                              'verbose'           : -1,
                          }
                       ),
 

  'XGBC': XGBClassifier(**{'tree_method'        : "gpu_hist" if CFG.gpu_switch == "ON" else "hist",
                           'objective'          : 'binary:logistic',
                           'random_state'       : CFG.state,
                           'colsample_bytree'   : 0.7,
                           'learning_rate'      : 0.0655,
                           'max_depth'          : 6,
                           'n_estimators'       : 1100,                         
                           'reg_alpha'          : 0.0001,
                           'reg_lambda'         : 2.25,
                           'min_child_weight'   : 18,
                           'early_stopping_rounds' : CFG.nbrnd_erly_stp,
                           'eval_metric'        : "auc",
                        }
                       ),
 
  "HGBC" : HGBC(loss              = 'log_loss',
                learning_rate     = 0.07,
                early_stopping    = True,
                max_iter          = 200,
                max_depth         = 6,
                min_samples_leaf  = 20,
                l2_regularization = 1.75,
                scoring           = myscorer,
                random_state      = CFG.state,
               ),
 
  "RFC" : RFC(n_estimators     = 150,
              criterion        = 'gini',
              max_depth        = 7,
              min_samples_leaf = 12,
              max_features     = 'sqrt',
              bootstrap        = True,
              oob_score        = True,
              n_jobs           = -1, 
              random_state     = CFG.state,
              verbose          = 0,
             ),
 
  "GBC" : GBC(loss           = 'log_loss',
              learning_rate  = 0.0525,
              n_estimators   = 250,
              subsample      = 0.45,
              min_samples_leaf = 18,
              max_depth      = 6,
              random_state   = CFG.state,
              n_iter_no_change = CFG.nbrnd_erly_stp,
             ),
 
  "LC"  : LC(penalty         = 'l2', 
                tol          = 0.0001, 
                C            = 0.01, 
                random_state = CFG.state,
                max_iter     = 1000,
                verbose      = 0,
               ),

};

In [ ]:
if CFG.ML == "Y":

    # 初始化模型输出
    if CFG.drop_nulls == "Y":
        methods      = list(Mdl_Master.keys());
    else:
        methods = ['XGBC', "LGBM1C", "LGBM2C", "CBC", "HGBC"];
    
    sel_cols  = \
    ['Source',
     'loc', 'vg', 'evg', 'ivg', 'n', 'v', 'l', 'd', 'i', 'e', 'b', 't',
     'lOCode', 'lOComment', 'lOBlank', 'locCodeAndComment', 'uniq_Op',
     'uniq_Opnd', 'total_Op', 'total_Opnd', 'branchCount', 
     'mean_bnv', 'mean_uniqOpOpend', 'mean_totOpOpend', 'mean_brcntvg'
    ];
    
    # （训练集结果）列标签：模型名称
    OOF_Preds = pd.DataFrame(columns = methods);
    # （测试集结果）索引：与sub保持一致；列标签：模型名称
    Mdl_Preds = pd.DataFrame(index = pp.sub_fl['id'], columns = methods);
    # （模型特征重要性）索引：除Source外的其他所有列
    FtreImp   = pd.DataFrame(index = sel_cols[1:], columns = methods);
    # （交叉检验每折分数）列标签：模型名称
    Scores    = pd.DataFrame(columns = methods);

    PrintColor(f"\n---> Selected model options- ");
    pprint(methods, depth = 1, width = 100, indent = 5);

In [ ]:
def TrainMdl(method:str):
    
    # 设定全局变量
    global Mdl_Master, Mdl_Preds, OOF_Preds, all_cv, FtreImp, Xtrain, ytrain, sel_cols; 
    
    # 若methed包含逻辑回归、支持向量机、K邻近、MLP模型
    if method in ["LC", "SVC", "KNNC", "MLPC"]:
        # 进行log变换、标准化变换、取模型
        model = Pipeline(steps = [('Log', FT(np.log1p)), 
                                  ("S", all_scalers[CFG.scl_method]), 
                                  ("M", Mdl_Master.get(method))
                                 ]);
    else:
        # 直接取模型
        model = Pipeline(steps = [("M", Mdl_Master.get(method))]); 
    # 取交叉检验方式
    cv        = all_cv.get(CFG.mdlcv_mthd);
    # 取测试集对应列
    Xt        = Xtest[sel_cols];
    
    if CFG.use_orig_allfolds == "Y":
        # 从Xtrain中选择特征并筛选出Source为'Competition'的行
        X    = Xtrain[sel_cols].query("Source == 'Competition'");
        # 获取与X的索引匹配的ytrain的子集
        y    = ytrain.loc[ytrain.index.isin(X.index)]; 
        # 创建数据框，其中包含Source为'Original'的行和特征
        Orig = pd.concat([Xtrain, ytrain], axis=1).query("Source == 'Original'")[sel_cols];
        
    elif CFG.use_orig_allfolds != "Y":
        # 直接使用Xtrain的特定列
        X,y = Xtrain[sel_cols], ytrain.copy(deep = True);
                
    # 初始化参数        
    test_preds = 0;
    oof_preds  = pd.DataFrame(); 
    scores     = [];
    ftreimp    = 0;
    cols_drop  = ['Source'];
    
    # 开始执行交叉检验
    for fold_nb, (train_idx, dev_idx) in enumerate(cv.split(X, y)):
        # 从训练集中删除目标（Source）列
        Xtr  = X.iloc[train_idx].drop(columns = cols_drop, errors = 'ignore');   
        # 从验证集中筛选出'Source'为'Competition'的行
        Xdev = X.iloc[dev_idx].loc[X.Source == "Competition"].\
        drop(columns = cols_drop, errors = 'ignore');
        # 取训练集对应目标列
        ytr  = y.loc[y.index.isin(Xtr.index)];
        # 取验证集对应目标列
        ydev = y.loc[y.index.isin(Xdev.index)];

        if CFG.use_orig_allfolds == "Y":
            # 将Xtr和ytr与Orig合并
            Xtr = pd.concat([Xtr, Orig.drop(columns = [CFG.target, 'Source'], errors = 'ignore')], 
                            axis = 0, ignore_index = True);
            ytr = pd.concat([ytr, Orig[CFG.target]], axis = 0, ignore_index = True);
            
        # 训练模型
        # ‘CBC’或'CBR'模型
        if method in ['CBC', 'CBR']:    
            model.fit(Xtr, ytr, 
                      M__eval_set = [(Xdev, ydev)], 
                      M__verbose = 0,
                      M__early_stopping_rounds = CFG.nbrnd_erly_stp,
                                           ); 
        # ‘LGBM’模型
        elif "LGBM" in method:
            model.fit(Xtr, ytr, 
                      M__eval_set = [(Xdev, ydev)], 
                      M__verbose = 0,
                      M__callbacks = [log_evaluation(0), 
                                      early_stopping(stopping_rounds = CFG.nbrnd_erly_stp, 
                                                     verbose = False,)
                                     ]
                     ); 
        # XGB模型
        elif "XGB" in method:
             model.fit(Xtr, ytr, M__eval_set = [(Xdev, ydev)], M__verbose = 0,);            
        
        # sklearn封装模型   
        else: 
            model.fit(Xtr, ytr); 
            
        # 收集预测结果和分数，并根据模型方法对Out-of-Fold（OOF）数据进行后处理
        if method.upper().endswith('R'):
            # 对于回归模型
            dev_preds = PostProcessPred(model.predict(Xdev), 
                                        post_process= CFG.pstprcs_train);
            test_preds = test_preds + \
            PostProcessPred(model.predict(Xt.drop(columns = cols_drop, errors = 'ignore')),
                            post_process= CFG.pstprcs_train); 
        else:
            # 对于分类模型，获取类别为1的概率
            dev_preds = model.predict_proba(Xdev)[:,1];
            test_preds = test_preds + \
            model.predict_proba(Xt.drop(columns = cols_drop, errors = 'ignore'))[:,1];            
        # AUC指标
        score = ScoreMetric(ydev.values.flatten(), dev_preds);
        scores.append(score); 
         
        # 将得分保存到Scores数据框中
        Scores.loc[fold_nb, method] = np.round(score, decimals= 6);
        # 将OOF预测值添加到oof_preds中
        oof_preds = pd.concat([oof_preds,
                               pd.DataFrame(index   = Xdev.index, 
                                            data    = dev_preds,
                                            columns = [method])
                              ],axis=0, ignore_index= False
                             );  
    
        oof_preds = pd.DataFrame(oof_preds.groupby(level = 0)[method].mean());
        oof_preds.columns = [method];
        
        try: 
            ftreimp += model["M"].feature_importances_;
        except: 
            ftreimp = 0;
    # 打印模型的平均交叉验证分数
    num_space = 10 - len(method);
    PrintColor(f"--> {method}{'-' * num_space} CV = {np.mean(scores):.6f}");
    del num_space;
    
    # 对OOF预测值进行后处理并保存到OOF_Preds中
    OOF_Preds[f'{method}'] = PostProcessPred(oof_preds.values.flatten(), CFG.pstprcs_train);
    
    # 如果交叉验证方法是'KF'或'SKF'
    if CFG.mdlcv_mthd in ['KF', 'SKF']:
        Mdl_Preds[f'{method}'] = test_preds.flatten()/ CFG.n_splits; 
        FtreImp[method]        = ftreimp / CFG.n_splits;
    else:
        Mdl_Preds[f'{method}'] = test_preds.flatten()/ (CFG.n_splits * CFG.n_repeats); 
        FtreImp[method]        = ftreimp / (CFG.n_splits * CFG.n_repeats);

In [ ]:
if CFG.ML == "Y":
    if CFG.ML == "Y": 
        PrintColor(f"\n{'-' * 25} ML model training {'-' * 25}\n", color = Fore.MAGENTA);
        for method in tqdm(methods, "ML models----"): 
            TrainMdl(method);
        clear_output();

        PrintColor(f"\n{'-' * 20} OOF CV scores across methods {'-' * 20}\n", 
                   color = Fore.MAGENTA);
        display(pd.concat([Scores.mean(axis = 0), Scores.std(axis = 0)], axis=1).\
                rename(columns = {0: 'Mean', 1: 'Std'}).T.\
                style.format(precision = 6).\
                background_gradient(subset = methods, cmap = 'cubehelix', axis=1)
               );    
    else:
        PrintColor(f"\nML models are not needed\n", color = Fore.RED);

In [ ]:
# 分析模型结果、特征重要性和校准曲线
if CFG.ML == "Y":
    fig, axes = plt.subplots(len(methods), 2, figsize = (25, len(methods) * 6),
                             gridspec_kw = {'hspace': 0.2, 'wspace': 0.2}, 
                             width_ratios= [0.7, 0.3],
                            );
    
    for i, col in enumerate(methods):
        try: 
            ax = axes[i,0];
        except: 
            ax = axes[0];
        FtreImp[col].plot.barh(ax = ax, color = '#0073e6');
        ax.set_title(f"{col} Importances", **CFG.title_specs);
        ax.set(xlabel = '', ylabel = '');

        try:
            ax = axes[i,1];
        except:
            ax = axes[1];
            
        Clb.from_predictions(ytrain[0:len(OOF_Preds)], OOF_Preds[col], n_bins= 20, ref_line = True,
                             **{'color': '#0073e6', 'linewidth': 1.2, 
                                'markersize': 3.75, 'marker': 'o', 'markerfacecolor': '#cc7a00'},
                             ax = ax);
        ax.set_title(f"{col} Calibration", **CFG.title_specs);
        ax.set(xlabel = '', ylabel = '',);
        ax.set_yticks(np.arange(0,1.01, 0.05), labels = np.round(np.arange(0,1.01, 0.05), 2), fontsize = 7.0);
        ax.set_xticks(np.arange(0,1.01, 0.05), 
                      labels = np.round(np.arange(0,1.01, 0.05), 2), 
                      fontsize = 7.0, 
                      rotation = 90
                     );
        ax.legend('');
            


    plt.tight_layout();
    plt.show();

In [ ]:
# 集合模型
if CFG.ensemble_req == "Y":
    # 使用optuna搜寻模型最优权重
    def Objective(trial):
        global OOF_Preds, all_cv, ytrain, methods;

        # 定义权重搜索空间
        weights  = [trial.suggest_float(f"M{n}", 0.0001, 0.9999, step = 0.001) \
                    for n in range(len(OOF_Preds[methods].columns))
                   ];

        # 计算集合模型分数
        scores = [];  
        cv     = all_cv[CFG.enscv_mthd];
        # 取各模型预测概率
        X,y    = OOF_Preds[methods], ytrain[0: len(OOF_Preds)];

        # 交叉检验
        for fold_nb, (train_idx, dev_idx) in enumerate(cv.split(X,y)):
            Xtr, Xdev = X.iloc[train_idx], X.iloc[dev_idx];
            ytr, ydev = y.loc[Xtr.index],  y.loc[Xdev.index];
            scores.append(ScoreMetric(ydev, np.average(Xdev, axis=1, weights = weights)));
        return np.mean(scores);
clear_output();

In [ ]:
# 开始堆叠模型
if CFG.ensemble_req == "Y":   
    PrintColor(f"\n{'-' * 20} Creating an Optuna Ensemble {'-' * 20}\n", 
               color = Fore.MAGENTA);   
    # 最大化目标值
    study = optuna.create_study(direction  = CFG.metric_obj, 
                                study_name = "OptunaEnsemble", 
                                sampler    = TPESampler(seed = CFG.state)
                               );
    study.optimize(Objective, 
                   n_trials          = CFG.ntrials, 
                   gc_after_trial    = True,
                   show_progress_bar = True
                  );
    weights       = study.best_params;
    clear_output();
    
    PrintColor(f"\n--> Post ensemble weights\n");
    pprint(weights, indent = 5, width = 10, depth = 1);
    PrintColor(f"\n--> Best ensemble CV score = {study.best_value :.6f}\n");
    
    # 使用集合模型在获得测试集结果
    sub_fl = pp.sub_fl.copy(deep = True);
    sub_fl[CFG.target] = np.average(Mdl_Preds[methods], weights = list(weights.values()),
                                    axis=1);
    
    PrintColor(f"\n--> Post ensemble test-set predictions\n");
    display(sub_fl.head(5).style.format(precision = 5)); 
      
    # 创建结果直方图
    fig, ax = plt.subplots(1,1, figsize = (8,5), gridspec_kw = {'hspace': 0.2, 'wspace': 0.2});
    _ = pd.DataFrame(np.arange(0, 1.01, 0.03), dtype = np.float16);

    sns.histplot(sub_fl[CFG.target], color = '#0039e6', bins = 'auto', ax = ax);
    ax.set_title(f"\nEnsemble predictions before post-processing\n", **CFG.title_specs);
    ax.set(xlabel = '', ylabel = '');
    ax.set_xticks(_.values.flatten(), labels = np.round(_.values.flatten(),2), rotation = 90);
   
    plt.tight_layout();
    plt.show();

    del _;

In [ ]:
if CFG.ensemble_req == "Y":
    # 保存文件并准备提交
    public_sub = pd.read_csv(f"/kaggle/input/ps-s3-e23-eda-model-en-es/submission.csv", index_col = "id");
    sub_fl[CFG.target] = sub_fl[CFG.target].values * 0.05 + public_sub[CFG.target].values * 0.95;
    del public_sub;

    PrintColor(f"\n---> Submission file after blending with public notebook\n");
    display(sub_fl.head(10).style.format(precision = 3));
    sub_fl.to_csv(f"Submission_V{CFG.version_nb}.csv", index = None);
    
if CFG.ML == "Y":  
    OOF_Preds.add_prefix(f"V{CFG.version_nb}_").to_csv(f"OOF_Preds_V{CFG.version_nb}.csv");
    Mdl_Preds.add_prefix(f"V{CFG.version_nb}_").to_csv(f"Mdl_Preds_V{CFG.version_nb}.csv"); 
    if isinstance(Scores, pd.DataFrame) == True:
        Scores.to_csv(f"Scores_V{CFG.version_nb}.csv");